# Face Attendance System

This notebook implements a facial recognition-based attendance system using FaceNet and MTCNN. The system recognizes registered students, marks their attendance, and displays their information on a user interface.

## Features
- Real-time face detection and recognition
- Student information display
- Attendance tracking with cooldown periods
- Firebase integration for storage and database
- Custom UI with different status modes

## Requirements
- OpenCV
- TensorFlow
- Firebase Admin SDK
- MTCNN
- cvzone
- FaceNet pre-trained model


## 1. Import Required Libraries

In [ ]:
import os
import pickle
import numpy as np
import cv2
import cvzone
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity

## 2. Initialize Firebase

Connect to Firebase for data storage and retrieval. You'll need your own service account credentials file.

In [ ]:
# Initialize Firebase with your credentials
cred = credentials.Certificate("service.json")  # Replace with your service account key
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://bigvision-22c68-default-rtdb.firebaseio.com/",  # Update with your database URL
    'storageBucket': "bigvision-22c68.firebasestorage.app"  # Update with your storage bucket
})

# Get storage bucket reference
bucket = storage.bucket()

## 3. Initialize Video Capture and UI Resources

In [ ]:
# Initialize video capture (0 for default camera, or specify another index)
cap = cv2.VideoCapture(0)  # Change to 1 if using external webcam
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height

# Load background image for UI
imgBackground = cv2.imread('resources/background2.png')

# Load mode images for different UI states
folderModePath = 'resources/new_modes'
modePathList = os.listdir(folderModePath)
imgModeList = []
for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath, path)))

print(f"Loaded {len(imgModeList)} mode images for UI")

## 4. Initialize Face Detection and Recognition Models

In [ ]:
# Initialize MTCNN for face detection
face_detector = MTCNN()
print("MTCNN face detector initialized")

# Load FaceNet model for face embedding generation
facenet_model_path = 'models/facenet_keras.h5'
if not os.path.exists(facenet_model_path):
    print(f"ERROR: FaceNet model not found at {facenet_model_path}")
    print("Please download the model from: https://github.com/nyoki-mtl/keras-facenet")
    # You would exit here in a script, but in a notebook let's just raise an exception
    raise FileNotFoundError(f"FaceNet model not found at {facenet_model_path}")

facenet_model = load_model(facenet_model_path)
print("FaceNet model loaded successfully")

## 5. Load Existing Encodings (if available)

In [ ]:
# Load the encoding file (or create empty lists if not exists)
print("Loading Encode File...")
encode_file_path = 'FaceNetEncodeFile.p'
if os.path.exists(encode_file_path):
    with open(encode_file_path, 'rb') as file:
        encodeListKnownWithIds = pickle.load(file)
    encodeListKnown, studentIds = encodeListKnownWithIds
    print(f"Encode File Loaded with {len(encodeListKnown)} face encodings")
else:
    print("No existing encode file found. Will create when enrolling new students.")
    encodeListKnown = []  # List to store face embeddings
    studentIds = []       # List to store corresponding student IDs

## 6. Define Constants and Configuration

In [ ]:
# Constants for timing (in frames, assuming ~25fps)
ACTIVE_DISPLAY_TIME = 5 * 25      # 5 seconds for "ACTIVE" screen
DETAILS_DISPLAY_TIME = 5 * 25     # 5 seconds for student details screen
MARKED_DISPLAY_TIME = 2 * 25      # 2 seconds for "MARKED" screen (green checkmark)
ALREADY_MARKED_DISPLAY_TIME = 3 * 25  # 3 seconds for "ALREADY MARKED" screen
COOLDOWN_PERIOD = 60              # 1 minute cooldown before allowing re-marking
SIMILARITY_THRESHOLD = 0.85       # Threshold for face match confidence (higher = stricter)

# Mode Types:
# 0: Scanning/Active mode
# 1: Student details mode (photo, ID, name)
# 2: Marked mode (green checkmark)
# 3: Already marked mode

# Initial states
modeType = 0           # Start in scanning mode
counter = 0            # Frame counter for timing
id = -1                # Current student ID
imgStudent = None      # Student image placeholder
studentInfo = None     # Student info placeholder
marked_students = {}   # Dictionary to track recently marked students {id: timestamp}
face_detected = False  # Flag for face detection status

## 7. Define Helper Functions

In [ ]:
def preprocess_face(face_img, required_size=(160, 160)):
    """Preprocess face image for FaceNet input
    
    Args:
        face_img: The face image to process
        required_size: Target size for the image (FaceNet expects 160x160)
        
    Returns:
        Preprocessed face image ready for embedding generation
    """
    face_img = cv2.resize(face_img, required_size)
    face_img = face_img.astype('float32')
    # Standardize pixel values
    mean, std = face_img.mean(), face_img.std()
    face_img = (face_img - mean) / std
    return face_img

def get_embedding(face_img):
    """Generate embedding vector from preprocessed face using FaceNet
    
    Args:
        face_img: Preprocessed face image
        
    Returns:
        128-dimensional embedding vector
    """
    # Expand dimensions to match the model's expected input
    samples = np.expand_dims(face_img, axis=0)
    # Get embeddings
    yhat = facenet_model.predict(samples)
    return yhat[0]

def detect_faces(img):
    """Detect faces using MTCNN and return face locations and aligned faces
    
    Args:
        img: Input frame from camera
        
    Returns:
        Tuple of (face_locations, aligned_faces)
    """
    # Convert to RGB for MTCNN
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Detect faces
    results = face_detector.detect_faces(rgb_img)
    
    face_locations = []
    aligned_faces = []
    
    for result in results:
        # Get face box coordinates
        x, y, w, h = result['box']
        # Handle negative values sometimes returned by MTCNN
        x, y = max(0, x), max(0, y)
        face_img = rgb_img[y:y+h, x:x+w]
        
        # Skip if face is too small
        if face_img.shape[0] < 20 or face_img.shape[1] < 20:
            continue
            
        # Process and align face
        try:
            # Convert back to BGR for consistency with OpenCV
            face_img = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
            # Preprocess for FaceNet
            processed_face = preprocess_face(face_img)
            # Add to results
            face_locations.append((y, x+w, y+h, x))  # Format: (top, right, bottom, left)
            aligned_faces.append(processed_face)
        except Exception as e:
            print(f"Error processing face: {e}")
            continue
            
    return face_locations, aligned_faces

def compare_faces(known_embeddings, face_embedding, threshold=SIMILARITY_THRESHOLD):
    """Compare a face embedding with a list of known embeddings
    
    Args:
        known_embeddings: List of known face embeddings
        face_embedding: Current face embedding to compare
        threshold: Similarity threshold (0-1)
        
    Returns:
        Tuple of (matches, similarities)
    """
    if len(known_embeddings) == 0:
        return [], []
    
    # Calculate similarity for each known face
    similarities = []
    for emb in known_embeddings:
        similarity = cosine_similarity([emb], [face_embedding])[0][0]
        similarities.append(similarity)
    
    # Create match results
    matches = [sim >= threshold for sim in similarities]
    
    return matches, similarities

def draw_centered_text(image, text, box_x, box_y, box_w, box_h, font_scale=0.6, thickness=2):
    """Draw text centered in a box area
    
    Args:
        image: Image to draw on
        text: Text to draw
        box_x, box_y: Top-left coordinates of the box
        box_w, box_h: Width and height of the box
        font_scale: Text size
        thickness: Text thickness
    """
    font = cv2.FONT_HERSHEY_COMPLEX
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_width, text_height = text_size
    x = box_x + (box_w - text_width) // 2
    y = box_y + (box_h + text_height) // 2 - 5  # fine-tuned for vertical alignment
    cv2.putText(image, text, (x, y), font, font_scale, (0, 0, 0), thickness)  # black color

## 8. Main Loop for Face Recognition and Attendance System

In [ ]:
print("Starting face attendance system with FaceNet...")

try:
    while True:
        # 1. Capture and prepare frame
        success, img = cap.read()
        if not success:
            print("Failed to get frame from camera")
            continue

        # Create a copy of the background for each frame
        imgBackground = cv2.imread('resources/background2.png')
        
        # Resize webcam feed to match the green box perfectly
        imgResized = cv2.resize(img, (655, 380))

        # Define top-left coordinate of the green box
        x, y = 137, 244

        # Paste webcam feed onto background
        imgBackground[y:y+380, x:x+655] = imgResized
        
        # Create a copy to work with for drawing
        imgBackgroundCopy = imgBackground.copy()

        # Update the right panel image based on current mode
        panelResized = cv2.resize(imgModeList[modeType], (438, 714))
        panel_x, panel_y = 899, 23
        imgBackgroundCopy[panel_y:panel_y+714, panel_x:panel_x+438] = panelResized

        # 2. Process face detection in scanning mode
        if modeType == 0:  # Scanning mode
            # Detect and align faces
            face_locations, processed_faces = detect_faces(img)
            
            # Reset face_detected flag
            face_detected = False
            
            if face_locations and processed_faces:
                for face_loc, processed_face in zip(face_locations, processed_faces):
                    # Get the embedding for the current face
                    face_embedding = get_embedding(processed_face)
                    
                    # Compare with known faces
                    matches, similarities = compare_faces(encodeListKnown, face_embedding)
                    
                    if any(matches):
                        # Find the best match
                        match_index = np.argmax(similarities)
                        
                        # If match confidence is good enough
                        if matches[match_index]:
                            # Draw rectangle around face
                            y1, x2, y2, x1 = face_loc
                            bbox = x + x1, y + y1, x2 - x1, y2 - y1
                            imgBackgroundCopy = cvzone.cornerRect(imgBackgroundCopy, bbox, rt=0)
                            
                            # Get student ID
                            id = studentIds[match_index]
                            face_detected = True
                            
                            # Check if student was recently marked (within cooldown period)
                            current_time = datetime.now()
                            if id in marked_students:
                                time_diff = (current_time - marked_students[id]).total_seconds()
                                
                                # If within cooldown period, show "Already Marked" screen
                                if time_diff < COOLDOWN_PERIOD:
                                    counter = 1
                                    modeType = 3  # Switch to "Already Marked" mode
                                    
                                    # Update right panel immediately for "Already Marked"
                                    panelResized = cv2.resize(imgModeList[modeType], (438, 714))
                                    imgBackgroundCopy[panel_y:panel_y+714, panel_x:panel_x+438] = panelResized
                                    
                                    print(f"Student ID {id} already marked - showing 'Already Marked' screen")
                                    continue
                            
                            # 3. Fetch student data
                            try:
                                studentInfo = db.reference(f'Students/{id}').get()
                                print(f"Retrieved student info for ID {id}")
                                
                                # Get student image from storage
                                try:
                                    blob = bucket.get_blob(f'images/{id}.jpeg')
                                    if blob:
                                        array = np.frombuffer(blob.download_as_string(), np.uint8)
                                        imgStudent = cv2.imdecode(array, cv2.COLOR_BGRA2BGR)
                                        print(f"Successfully loaded student image for ID {id}")
                                    else:
                                        print(f"No image found for student ID {id}")
                                        imgStudent = None
                                except Exception as e:
                                    print(f"Error loading student image: {e}")
                                    imgStudent = None
                                
                                # 4. Update attendance in database
                                if studentInfo and 'last_attendance_time' in studentInfo:
                                    datetimeObject = datetime.strptime(studentInfo['last_attendance_time'], 
                                                                    "%Y-%m-%d %H:%M:%S")
                                    secondsElapsed = (datetime.now() - datetimeObject).total_seconds()
                                    
                                    # Update attendance if enough time has passed since last mark
                                    if secondsElapsed > 30:  # Database cooldown check
                                        ref = db.reference(f'Students/{id}')
                                        studentInfo['total_attendance'] += 1
                                        ref.child('total_attendance').set(studentInfo['total_attendance'])
                                        ref.child('last_attendance_time').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                                        
                                        # Record this student as recently marked
                                        marked_students[id] = datetime.now()
                                        print(f"Updated attendance for student ID {id}")
                                        
                                        # Switch to "Student Details" mode (photo, ID, name)
                                        counter = 1
                                        modeType = 1
                                        
                                        # Update right panel immediately for "Student Details"
                                        panelResized = cv2.resize(imgModeList[modeType], (438, 714))
                                        imgBackgroundCopy[panel_y:panel_y+714, panel_x:panel_x+438] = panelResized
                                    else:
                                        # Student was marked too recently (database perspective)
                                        counter = 1
                                        modeType = 3  # Show as "Already Marked"
                                        
                                        # Update right panel immediately
                                        panelResized = cv2.resize(imgModeList[modeType], (438, 714))
                                        imgBackgroundCopy[panel_y:panel_y+714, panel_x:panel_x+438] = panelResized
                                else:
                                    print(f"No attendance data for student ID {id}")
                            except Exception as e:
                                print(f"Error processing student data: {e}")
                                counter = 0
                                modeType = 0
        
        # 5. Handle mode transitions based on counters
        if counter > 0:
            counter += 1
            
            # After showing student details screen for specified time, transition to marked screen
            if modeType == 1 and counter >= DETAILS_DISPLAY_TIME:
                print("Transitioning from details to marked mode")
                modeType = 2  # Switch to marked mode (green checkmark)
                counter = 1  # Reset counter for marked mode
                
                # Update the right panel immediately for marked
                panelResized = cv2.resize(imgModeList[modeType], (438, 714))
                imgBackgroundCopy[panel_y:panel_y+714, panel_x:panel_x+438] = panelResized
            
            # When in student details mode, display student information
            if modeType == 1:
                if studentInfo:
                    # Display student information
                    try:
                        # Draw ID
                        if id is not None:
                            draw_centered_text(imgBackgroundCopy, str(id), 1025, 555, 100, -170)

                        # Draw Name
                        if 'name' in studentInfo:
                            draw_centered_text(imgBackgroundCopy, studentInfo['name'], 1025, 555, 200, 40)

                        # Display student image
                        if imgStudent is not None and isinstance(imgStudent, np.ndarray) and imgStudent.size > 0:
                            try:
                                imgStudentResized = cv2.resize(imgStudent, (216, 216))
                                imgBackgroundCopy[150:150 + 216, 1009:1009 + 216] = imgStudentResized
                            except Exception as e:
                                print(f"Error displaying student image: {e}")
                    except Exception as e:
                        print(f"Error displaying student info: {e}")
            
            # After showing "Already Marked" for specified time, return to scanning mode
            elif modeType == 3 and counter >= ALREADY_MARKED_DISPLAY_TIME:
                counter = 0
                modeType = 0
                studentInfo = None
                imgStudent = None
            
            # After showing "Marked" screen (checkmark) for specified time, return to scanning mode
            elif modeType == 2 and counter >= MARKED_DISPLAY_TIME:
                counter = 0
                modeType = 0
                studentInfo = None
                imgStudent = None
        
        # 6. Clean up expired entries in marked_students dictionary
        current_time = datetime.now()
        expired_ids = []
        for student_id, mark_time in marked_students.items():
            if (current_time - mark_time).total_seconds() > COOLDOWN_PERIOD:
                expired_ids.append(student_id)
        
        for student_id in expired_ids:
            del marked_students[student_id]
        
        # 7. Display the UI
        cv2.imshow("Face Attendance System", imgBackgroundCopy)
        
        # Check for key press (ESC to exit)
        if cv2.waitKey(1) == 27:
            break

except KeyboardInterrupt:
    print("\nAttendance system stopped by user")
except Exception as e:
    print(f"\nError in attendance system: {e}")
finally:
    # Clean up resources
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released successfully")